# Walmart : predict weekly sales

In [95]:
!pip install plotly

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "iframe_connected" # to be replaced by "iframe" if working on JULIE

In [2]:
walmart = pd.read_csv("Walmart_Store_sales.csv")

In [3]:
walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [4]:
walmart.describe(include = 'all') #Dataset description

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [5]:
walmart.isna().sum() #Counting missing values in each column

Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

In [6]:
walmart['Holiday_Flag'].value_counts() #Looking at holiday flag column

0.0    127
1.0     11
Name: Holiday_Flag, dtype: int64

In [7]:
walmart.drop('Holiday_Flag', axis = 1, inplace = True) #As Holiday flag column is mostly null values.

In [8]:
display(100*walmart.isna().sum()/walmart.shape[0]) #Getting proportions of missing values

Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

#### Missing dates will be dropped since we are looking for informations where date is a critical parameter.

In [9]:
walmart.dropna(subset = ['Date'], axis = 0, how = 'all', inplace =True)
walmart.isna().sum()

Store            0
Date             0
Weekly_Sales    14
Temperature     14
Fuel_Price      13
CPI             10
Unemployment    12
dtype: int64

In [10]:
#Let's make sure it's chronological.
walmart["Date"] = pd.to_datetime(walmart["Date"])
walmart = walmart.sort_values(by="Date")

#### Let's look at our target : Weekly Sales

In [11]:
px.histogram(data_frame= walmart, x = 'Date' , y = 'Weekly_Sales', 
             color = 'Store',
             nbins = 60,
            title = 'Weekly Sales by Date')

#### We could have drop lines where weekly sales values are missing but as we don't have that many rows, we will fill missing values by mean.

In [12]:
sales_mean = walmart['Weekly_Sales'].mean()

In [13]:
#Replacing missing values by sales mean.
walmart['Weekly_Sales'] = walmart['Weekly_Sales'].fillna(sales_mean)

In [14]:
walmart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132 entries, 14 to 22
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         132 non-null    float64       
 1   Date          132 non-null    datetime64[ns]
 2   Weekly_Sales  132 non-null    float64       
 3   Temperature   118 non-null    float64       
 4   Fuel_Price    119 non-null    float64       
 5   CPI           122 non-null    float64       
 6   Unemployment  120 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 8.2 KB


Dropping outlayers

In [15]:
#Drop unemployment outlayers
mask = walmart['Weekly_Sales'] < (walmart['Weekly_Sales'].mean() + 3*walmart['Weekly_Sales'].std()) #3 standard deviations
walmart = walmart.loc[mask,:]

In [16]:
#Drop temperature outlayers
mask1 = walmart['Temperature'] < (walmart['Temperature'].mean() + 3*walmart['Temperature'].std()) 
walmart = walmart.loc[mask1,:]

#Drop fuelprice outlayers
mask2 = walmart['Fuel_Price'] < (walmart['Fuel_Price'].mean() + 3*walmart['Fuel_Price'].std())
walmart = walmart.loc[mask2,:]

#Drop cpi outlayers
mask3 = walmart['CPI'] < (walmart['CPI'].mean() + 3*walmart['CPI'].std())
walmart = walmart.loc[mask3,:]

#Drop unemployment outlayers
mask3 = walmart['Unemployment'] < (walmart['Unemployment'].mean() + 3*walmart['Unemployment'].std())
walmart = walmart.loc[mask3,:]

In [17]:
walmart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 14 to 22
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         88 non-null     float64       
 1   Date          88 non-null     datetime64[ns]
 2   Weekly_Sales  88 non-null     float64       
 3   Temperature   88 non-null     float64       
 4   Fuel_Price    88 non-null     float64       
 5   CPI           88 non-null     float64       
 6   Unemployment  88 non-null     float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 5.5 KB


In [18]:
walmart = walmart.drop('Date', axis = 1) #We won't use date column to train our model
walmart = walmart.drop('Store', axis = 1)

In [19]:
walmart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 14 to 22
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Weekly_Sales  88 non-null     float64
 1   Temperature   88 non-null     float64
 2   Fuel_Price    88 non-null     float64
 3   CPI           88 non-null     float64
 4   Unemployment  88 non-null     float64
dtypes: float64(5)
memory usage: 4.1 KB


In [20]:
walmart.isna().sum() #No more missing values

Weekly_Sales    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

### Some more visualizations

In [52]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Unemployment", "Fuel price",  "Temperature"))

fig.add_trace(go.Scatter(x = walmart['Unemployment'], y = walmart['Weekly_Sales'], mode='markers',), row=1, col=1)

fig.add_trace(go.Histogram(x = walmart['Fuel_Price'], y = walmart['Weekly_Sales'], nbinsx = 20), row=2, col=1)

fig.add_trace(go.Histogram(x = walmart['Temperature'], y = walmart['Weekly_Sales'], nbinsx = 15), row=3, col=1)

fig.update_layout(height=600, width=800, title_text="Unemployment, Fuel price and Temp impact on Sales")

fig.show()

# Pre processing

In [22]:
print("Separating labels from features...")
target_name = 'Weekly_Sales'
Y = walmart.loc[:,target_name]
X = walmart.loc[:,[c for c in walmart.columns if c!=target_name]] # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
14    8.292073e+05
20    5.611451e+05
99    2.018315e+06
47    1.549019e+06
84    1.257271e+06
Name: Weekly_Sales, dtype: float64

    Temperature  Fuel_Price         CPI  Unemployment
14        60.07       2.853  126.234600         6.885
20        38.26       2.725  189.704822         8.963
99        78.82       2.814  126.139200         7.951
47        66.25       2.958  132.521867         8.099
84        76.14       2.577  214.894576         6.315



In [67]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)
print("...Done.")
print()

...Done.



In [68]:
print("Converting pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values
print("...Done")

print(X_train[0:5,:])
print(X_test[0:2,:])
print()
print(Y_train[0:5])
print(Y_test[0:2])

Converting pandas DataFrames to numpy arrays...
...Done
[[ 69.8         4.069     134.8551613   7.658    ]
 [ 33.26        3.789     133.9587419   7.771    ]
 [ 76.14        2.577     214.894576    6.315    ]
 [ 50.49        2.854     204.201755    8.187    ]
 [ 39.3         3.936     197.7227385   8.09     ]]
[[ 48.29        3.75      197.4133259   6.162    ]
 [ 43.95        3.828     192.831317    6.339    ]]

[ 695396.19       1266564.94       1257271.13991525 1257271.13991525
  457340.06      ]
[485095.41 435397.19]


In [104]:
preprocessor = StandardScaler(copy = False)

In [105]:
preprocessor.fit_transform(X_train)
print(X_train[0:5,:])

[[ 0.50142409  1.48517414 -1.43708884  0.31879626]
 [-1.5318023   0.95639122 -1.46144204  0.43961562]
 [ 0.85420612 -1.33248343  0.73735706 -1.11713655]
 [-0.57305871 -0.80936604  0.44686317  0.88440196]
 [-1.19571342  1.23400225  0.27084651  0.78068976]]


In [106]:
preprocessor.transform(X_test)
print(X_test[0:5,:])

[[-0.69547519  0.88273931  0.26244064 -1.28072383]
 [-0.93696951  1.03004312  0.13796034 -1.0914758 ]
 [ 0.30388841 -0.61296095 -1.5004779   0.79031254]
 [ 1.3594523  -1.25316599  0.64696448  0.45672279]
 [ 1.23147144  0.51259127  0.85079694 -0.99952477]]


### Training - testing basic model

In [107]:
# Train model
print("Training model...")
regressor = LinearRegression() 
regressor.fit(X_train, Y_train)
print("...Done.")

Training model...
...Done.


In [108]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print(Y_train_pred[0:5])
print()

Predictions on training set...
[1298068.79925654 1307301.34261346 1006982.88240021 1170066.85930128
 1096159.34973289]



In [109]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print(Y_test_pred[0:5])
print()

Predictions on test set...
[ 950021.3714478   973224.03366774 1426024.74181034 1149910.59428076
  932449.71776439]



In [110]:
print("R2-score on train set : ",r2_score(Y_train, Y_train_pred))
print("R2-score on test set : ", r2_score(Y_test, Y_test_pred))

R2-score on train set :  0.07003413474649633
R2-score on test set :  0.06038930095476669


In [119]:
#Here are the correlation coefficients
coef = pd.DataFrame(data = regressor.coef_, index = X.columns, columns=['Coefficients'])
coef
#What we ca

,Coefficients
Temperature,12250.547124
Fuel_Price,-39846.287851
CPI,-133865.059061
Unemployment,81200.564583


### Ridge regularization

In [114]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
ridge = Ridge()

params = {
    'alpha': [10**a for a in range(100)] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(ridge, param_grid = params, cv = KFold(n_splits=3, shuffle=True, random_state=2)) # cv : the number of folds to be used for CV, we use Kfold here to shuffle the dataset
gridsearch.fit(X_train, Y_train)

print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

...Done.
Best hyperparameters :  {'alpha': 1000}
Best R2 score :  -0.005467568076498693


##### The model gives bad predictions. We could collect more data to improve predictions. In this case, when performing Gridsearch cross validation, shuffling was necessary to optimize scores (lowering the bias).